# Course Project - Paolo Teta & Ralfs Zangis
---

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
#!pip install tensorflow-gpu==2.0

import tensorflow as tf # import tensorflow under the alias "tf"
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

2.3.0
2.4.0


In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

---
# 1. Get the data

In [4]:
data = spark.read.option("header", True).option("inferSchema", "true").csv("./dataset.csv")

---
# 2. Look at the data

In [6]:
data.printSchema()
print((data.count(), len(data.columns)))

root
 |-- emotion: integer (nullable = true)
 |-- pixels: string (nullable = true)
 |-- usage: string (nullable = true)

(35887, 3)


In [7]:
data.show(5)

+-------+--------------------+--------+
|emotion|              pixels|   usage|
+-------+--------------------+--------+
|      0|70 80 82 72 58 58...|Training|
|      0|151 150 147 155 1...|Training|
|      2|231 212 156 164 1...|Training|
|      4|24 32 36 30 32 23...|Training|
|      6|4 0 0 0 0 0 0 0 0...|Training|
+-------+--------------------+--------+
only showing top 5 rows



In [9]:
data.filter(col("emotion") == 0).count()

4953

In [21]:
data.groupBy(col("emotion")).count().sort(asc("emotion")).show()
print(f"Total number of records in the dataset = {data.count()}")

+-------+-----+
|emotion|count|
+-------+-----+
|      0| 4953|
|      1|  547|
|      2| 5121|
|      3| 8989|
|      4| 6077|
|      5| 4002|
|      6| 6198|
+-------+-----+

Total number of records in the dataset = 35887


---
# 3. 